In [2]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm

season_dirs = {
    "Season 1": "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/lucknow_airshed/rgb",
    "Season 2": "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/seasons2_lucknow_airshed/images",
    "Season 3": "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/seasons3_lucknow_airshed/images",
    "Season 4": "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/seasons4_lucknow_airshed/images",
}

results = {}

for season, img_dir in season_dirs.items():
    image_paths = glob(os.path.join(img_dir, "*.png")) + glob(os.path.join(img_dir, "*.jpg"))
    means = []
    stds = []

    for path in tqdm(image_paths, desc=f"Processing {season}"):
        img = cv2.imread(path)  # Reads in BGR
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        means.append(np.mean(img, axis=(0, 1)))      # RGB mean
        stds.append(np.std(img, axis=(0, 1)))        # RGB std

    season_mean = np.mean(means, axis=0)
    season_std = np.mean(stds, axis=0)
    results[season] = {
        "mean_rgb": season_mean,
        "std_rgb": season_std
    }

# Print the results in a markdown-style table
print("| Season   | Mean R | Mean G | Mean B | Std R | Std G | Std B |")
print("|----------|--------|--------|--------|--------|--------|--------|")
for season, stats in results.items():
    m = stats["mean_rgb"]
    s = stats["std_rgb"]
    print(f"| {season} | {m[0]:.2f} | {m[1]:.2f} | {m[2]:.2f} | {s[0]:.2f} | {s[1]:.2f} | {s[2]:.2f} |")


Processing Season 1:   0%|          | 0/5184 [00:00<?, ?it/s]

Processing Season 4: 100%|██████████| 5184/5184 [00:09<00:00, 559.54it/s]

| Season   | Mean R | Mean G | Mean B | Std R | Std G | Std B |
|----------|--------|--------|--------|--------|--------|--------|
| Season 1 | 76.40 | 74.68 | 63.98 | 35.31 | 31.73 | 31.10 |
| Season 2 | 89.17 | 79.77 | 69.00 | 41.85 | 35.83 | 34.78 |
| Season 3 | 96.73 | 89.76 | 79.57 | 46.66 | 42.03 | 41.17 |
| Season 4 | 90.11 | 86.25 | 77.93 | 39.09 | 34.59 | 32.71 |


In [3]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
from skimage.measure import shannon_entropy
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray

def colorfulness(img):
    # Assumes img is RGB, uint8
    rg = np.abs(img[..., 0].astype(np.float32) - img[..., 1].astype(np.float32))
    yb = np.abs(0.5 * (img[..., 0].astype(np.float32) + img[..., 1].astype(np.float32)) - img[..., 2].astype(np.float32))
    std_rg, std_yb = np.std(rg), np.std(yb)
    mean_rg, mean_yb = np.mean(rg), np.mean(yb)
    return np.sqrt(std_rg ** 2 + std_yb ** 2) + 0.3 * np.sqrt(mean_rg ** 2 + mean_yb ** 2)

def edge_density(img):
    edges = cv2.Canny(img, 100, 200)
    return np.sum(edges > 0) / edges.size

def glcm_contrast(img_gray):
    # Reduce grayscale to 8 bits for GLCM
    img_gray = (img_gray * 255).astype(np.uint8)
    glcm = graycomatrix(img_gray, [1], [0], 256, symmetric=True, normed=True)
    return graycoprops(glcm, 'contrast')[0, 0]

season_names = {
    "Season 1": "Winter",
    "Season 2": "Pre",
    "Season 3": "Monsoon",
    "Season 4": "Post"
}
season_dirs = {
    "Season 1": "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/lucknow_airshed/rgb",
    "Season 2": "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/seasons2_lucknow_airshed/images",
    "Season 3": "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/seasons3_lucknow_airshed/images",
    "Season 4": "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/seasons4_lucknow_airshed/images",
}

results = {}

for season, img_dir in season_dirs.items():
    image_paths = glob(os.path.join(img_dir, "*.png")) + glob(os.path.join(img_dir, "*.jpg"))
    means, stds = [], []
    colorfulnesses, entropies, contrasts, brightnesses, edgedensities = [], [], [], [], []

    for path in tqdm(image_paths, desc=f"Processing {season}"):
        img = cv2.imread(path)
        if img is None:
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        means.append(np.mean(img_rgb, axis=(0, 1)))
        stds.append(np.std(img_rgb, axis=(0, 1)))
        colorfulnesses.append(colorfulness(img_rgb))
        entropies.append(shannon_entropy(img_rgb))
        img_gray = rgb2gray(img_rgb)
        contrasts.append(glcm_contrast(img_gray))
        brightnesses.append(np.mean(0.2126 * img_rgb[..., 0] + 0.7152 * img_rgb[..., 1] + 0.0722 * img_rgb[..., 2]))
        edgedensities.append(edge_density(img_rgb))

    results[season_names[season]] = {
        "mean_rgb": np.mean(means, axis=0),
        "std_rgb": np.mean(stds, axis=0),
        "colorfulness": np.mean(colorfulnesses),
        "entropy": np.mean(entropies),
        "glcm_contrast": np.mean(contrasts),
        "brightness": np.mean(brightnesses),
        "edge_density": np.mean(edgedensities)
    }

# Print the results in a markdown-style table
print("| Season  | Mean R | Mean G | Mean B | Std R | Std G | Std B | Colorfulness | Entropy | GLCM Contrast | Brightness | Edge Density |")
print("| ------- | ------ | ------ | ------ | ----- | ----- | ----- | ------------ | ------- | ------------- | ---------- | ------------ |")
for season, stats in results.items():
    m, s = stats["mean_rgb"], stats["std_rgb"]
    print(f"| {season} | {m[0]:.2f} | {m[1]:.2f} | {m[2]:.2f} | "
          f"{s[0]:.2f} | {s[1]:.2f} | {s[2]:.2f} | "
          f"{stats['colorfulness']:.2f} | {stats['entropy']:.2f} | {stats['glcm_contrast']:.2f} | "
          f"{stats['brightness']:.2f} | {stats['edge_density']:.4f} |")


Processing Season 4: 100%|██████████| 5184/5184 [00:27<00:00, 190.37it/s]

| Season  | Mean R | Mean G | Mean B | Std R | Std G | Std B | Colorfulness | Entropy | GLCM Contrast | Brightness | Edge Density |
| ------- | ------ | ------ | ------ | ----- | ----- | ----- | ------------ | ------- | ------------- | ---------- | ------------ |
| Winter | 76.40 | 74.68 | 63.98 | 35.31 | 31.73 | 31.10 | 16.87 | 6.95 | 225.32 | 74.27 | 0.2476 |
| Pre | 89.17 | 79.77 | 69.00 | 41.85 | 35.83 | 34.78 | 20.20 | 7.17 | 290.22 | 80.99 | 0.2722 |
| Monsoon | 96.73 | 89.76 | 79.57 | 46.66 | 42.03 | 41.17 | 16.80 | 7.31 | 353.00 | 90.51 | 0.2709 |
| Post | 90.11 | 86.25 | 77.93 | 39.09 | 34.59 | 32.71 | 17.64 | 7.05 | 251.76 | 86.47 | 0.2443 |
